# Bootstrapping with Coulomb Potential

In this code, we demonstrate radial Coulomb potential with $\hbar = 1$ and $m_e = 1$. We begin with

\begin{align*}
    H &= \frac{P_r^2}{2m_e} + \frac{l(l+1)}{2m_er^2}\hbar^2 + V(r)\\
    V(r) &= -\frac{e^2}{4\pi\epsilon_0r} = -\frac{k}{r}\\
    [r,P_r] &= i
\end{align*}

We see that the Hamiltonian is similar to 1-D Harmonic potential, with additional angular term $\frac{l(l+1)}{2r^2}$. So we just need to consider the additional commutation relation of the additional angular term. (Denote $P_r=P$ in the following)

\begin{align*}
    [H, r^{s}P] &= 0 = 
    -\frac{1}{2}s(s-1)\langle r^{s-2}P\rangle 
    -is\langle r^{s-1}P^2\rangle
    +i\langle r^{s}V^\prime(r)\rangle
    -il(l+1)\langle r^{s-3}\rangle\\
    
    [H, r^{s-1}] &= 0 =
    -\frac{1}{2}(s-1)(s-2)\langle r^{s-3}\rangle
    -i(s-1)\langle r^{s-2}P\rangle
\end{align*}

With the same trick and the same algebraic calcukation, we get the final recursion relation
\begin{equation*}
    0 = 2sE\langle r^{s-1}\rangle + \frac{1}{4}s(s-1)(s-2)\langle r^{s-3}\rangle 
    - \langle r^s V^\prime (r)\rangle - 2s\langle r^{s-1}V(r)\rangle -(s-1)l(l+1)\langle r^{s-3}\rangle
\end{equation*}

**Note according to the author, $l=0$ case will be failed with  Bootstrapping, the reason in not clear yet.**

In [1]:
import os, time
import numpy as np
import sympy as sp
from bootstrap_sympy import sympy_solve_intervals, plot_energy_interval

import matplotlib.pyplot as plt
import seaborn as sns

## Recursion relation for 3-d Coulomb potential

Plug in $V(r)=-\frac{k}{r}$ into the recursion relation, we get

\begin{equation*}
    8(s+1)E\langle r^s\rangle = -4(2s+1)k\langle r^{s-1}\rangle
    + 4sl(l+1)\langle r^{s-2}\rangle - s(s+1)(s-1)\langle r^{s-2}\rangle
\end{equation*}

Take $s=0$ we get $\langle r^{-1}\rangle=-\frac{2E}{k}$ . If $s>0$

\begin{equation*}
    -E\langle r^s\rangle = \frac{k(2s+1)}{2(s+1)}\langle r^{s-1}\rangle
    + [\frac{s(s-1)}{8} - \frac{sl(l+1)}{2(s+1)}]\langle r^{s-2}\rangle
\end{equation*}

$\langle r^0\rangle=1$ by normalization. Take $s=1$, we get $\langle r^1\rangle=-\frac{3k}{4E}-\frac{l(l+1)}{2k}$

In [19]:
class ColumbPotentialMatrix:
    def __init__(self, N, angular_momentum=True):
        self.N = N # maximum size of submatrix
        self.k = sp.symbols('k') # constant of harmonic potential
        self.E = sp.symbols('E') # eigen-energy to be solved (Note the true energy is 1/E)
        self.l = sp.symbols('l') # angular momentum quantum number
        self.angular_momentum = angular_momentum
        
    def evaluate(self):
        self.rs_list = [] # list of r^s, the expectation value of position operator r to the s power
        for i in range(2*(self.N-1)+1):
            if i >= 2:
                self.rs_list.append(self.rs_recursion(s=i, rs_1=self.rs_list[i-1], rs_2=self.rs_list[i-2]))
            else:
                self.rs_list.append(self.rs_recursion(s=i))

    def rs_recursion(self, s, rs_1=None, rs_2=None):
        # find the <r^s> with recursion relation
        E = -1/self.E
        k = self.k
        l = self.l
        if s == 0:
            return 1
        elif s == 1:
            result = -sp.Rational(3,4) * k / E
            if self.angular_momentum:
                result += -sp.Rational(1,2) * l*(l+1) / k
            return result
        else:
            if rs_1 == None or rs_2 == None:
                rs_1 = self.rs_recursion(s-1)
                rs_2 = self.rs_recursion(s-2)
            result  = k * sp.Rational(2*s+1,2*s+2) * rs_1 * (-1/E)
            result += sp.Rational(s*(s-1),8) * rs_2 * (-1/E)
            if self.angular_momentum:
                result += -sp.Rational(s,2*s+2) * l*(l+1) * rs_2 * (-1/E)
            return sp.simplify(result)

    def submatrix(self, L):
        # L*L matrix's [i,j] element = <r^(i+j)>
        return sp.Matrix([[self.rs_list[i+j] for j in range(L)] for i in range(L)])

In [25]:
cp_config = {
    'x_inf' : 0,
    'x_sup' : 10,
    'round' : 10,
    'plot_step' : 2,
    'threshold' : 1e-2,
    'initial_interval' : sp.Interval(0, sp.oo),
}

cp_matrix = ColumbPotentialMatrix(N=cp_config['round'], angular_momentum=True)
cp_matrix.k = 1
cp_matrix.l = 1
cp_matrix.evaluate()

energy_intervals, confirmed_intervals = sympy_solve_intervals(cp_matrix, cp_config, mode='Poly', keep=False)

Now calculating determinant size 1x1
Time cost = 0.00
Survived interval = Interval(0, oo)

Now calculating determinant size 2x2
Time cost = 0.01
Survived interval = Interval(2.47213595499958, oo)

Now calculating determinant size 3x3
Time cost = 0.03
Survived interval = Interval(4.80974570283234, oo)

Now calculating determinant size 4x4
Time cost = 0.12
Survived interval = Interval(6.39879718888275, oo)

Now calculating determinant size 5x5
Time cost = 0.38
Survived interval = Interval(6.91519461524601, oo)

Now calculating determinant size 6x6
Time cost = 0.97
Survived interval = Interval(6.96903203123286, oo)

Now calculating determinant size 7x7
Time cost = 1.92
Survived interval = Union(Interval(7.04676432911461, 8.37674826680287), Interval(14.6874946423708, oo))

Now calculating determinant size 8x8
Time cost = 4.64
Survived interval = Union(Interval(7.27604718919460, 8.11009662762563), Interval(15.3227434875546, oo))

Now calculating determinant size 9x9
Time cost = 11.94
Surviv